# About the code 

This piece of code is divided into 2 parts <br>
---> 1. Make Chunks based on the Postal Codes and Table of contents extracted from the HTML: Here we are preparing a dataframe of all the file names that we read and try to divinde them into chunks based on the postal codes and the presence of outlines. <br>
---> ---> 1 a. 

In [2]:
import PyPDF2 as p2
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
import pandas as pd
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import json
import numpy as np
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [3]:
#define the paths of the PDF scrapped 

path = 'F:/Environmental Baseline Data/Version 3/Data/PDF'
pdfs = os.listdir(path)
pdf_paths = [str(path) + '/' + str(x) for x in pdfs]
len(pdf_paths)

1329

# 1. Make Chunks based on the Postal Codes and Outlines 

This part of the code will generate a csv file with all the PDF nammes with the fillowing details: <br> - 'file_nos': just a number to indicate the file
<br> - 'file_names': the name of the file (Postal Code.pdf)
<br> - 'file_paths' : the path of the pdf file
<br> - 'int_postals' : the integer value of the postal code
<br> - 'chunk_nos': chunk_no based on the consectutive Postal Code
    

### 1 a. Postal to Int Functions 

In [4]:
def postal_to_int(Postal):
    multiplier = 1
    odd = 1
    Postal_Int_Word = 0
    Error_Message = "Invalid Postal Code"
    
    # Reversing the string 
    Postal = Postal[::-1] 
    
    # Chcek for the length of the Postal Code
    if not(len(Postal)==6):
        return(Error_Message)
    
    for letter in list(str.strip(Postal)):
        if odd%2 == 1:
            # Check if it is an interger or not   
            # If this is not an integer then exit
            if not (str.isnumeric(letter)):
                return(Error_Message)
            
            Postal_Int_Letter = multiplier * int(letter)
            Postal_Int_Word = Postal_Int_Word + Postal_Int_Letter
            
            odd = odd + 1
            multiplier = multiplier * 10
            # print(letter, Postal_Int_Letter)
            # print(Postal_Int_Word)
        else:
            # Check if it is a character or not
            # If this is not a character then exit
            if not (str.isalpha(letter)):
                return(Error_Message)
            
            # ord('A') = 65
            Postal_Int_Letter = multiplier * (ord(letter) - 65)
            Postal_Int_Word = Postal_Int_Word + Postal_Int_Letter
            
            odd = odd + 1
            multiplier = multiplier * 26
            # print(letter, Postal_Int_Letter)
            # print(Postal_Int_Word)
            
    return Postal_Int_Word

In [5]:
# never used integer to Postal but I guess it would be useful 
# Also useful in validating if Postal to Int Function works 

def int_to_postal(Postal_Int_Word):
    Postal_Int_Word_Rem = Postal_Int_Word
    Postal = ""
    odd = 0
    Error_Message = "Invalid Postal Code"
    
    # Reversing the string 
    Postal = Postal[::-1] 
 
    while odd < 6:
        if odd%2 == 0:
            Postal_Int_Letter = Postal_Int_Word_Rem % 10
            Postal = Postal + str(Postal_Int_Letter)
            Postal_Int_Word_Rem = int(Postal_Int_Word_Rem/10)
            odd = odd + 1

        else:
            Postal_Int_Letter = (Postal_Int_Word_Rem % 26) +65
            Postal = Postal + chr(Postal_Int_Letter)
            Postal_Int_Word_Rem = int(Postal_Int_Word_Rem/26)
            odd = odd + 1
            
    # Accounts for 7 digit Postal Codes
    if Postal_Int_Word_Rem > 0:
        return(Error_Message)
        
    return Postal[::-1]
  

In [6]:
file_nos = []
file_names = []
file_paths = []
int_postals = []
file_no = 1

for pdf_path in pdf_paths:
    # additional check to get rid of the string "Thumb"
    if 'Thum' in pdf_path:
        continue
    
    file_nos.append(file_no)
    file_name = str(pdf_path.split('/')[-1])
    file_names.append(file_name)
    file_paths.append(pdf_path)
    
    postal = file_name[:-4]
    int_postal = postal_to_int(postal)
    int_postals.append(int_postal)
    
len(file_nos)
len(file_names)
len(file_paths)
len(int_postals)

1328

1328

1328

1328

### 1 c. Check if TOC is present 

In [7]:
# Getting the path of the TOC present
path_Texts = []

for path_pdf in pdf_paths:
    if 'Thumb' in path_pdf:
        continue
    path_Text = path_pdf.replace(" .pdf", "")
    if ".pdf" in path_Text:
        path_Text = path_pdf.replace(".pdf", "")
    #print(path_Text)
    path_Text = path_Text.replace("/PDF", "/HTML Images and Tables")
    path_Text = path_Text + "/table-of-contents.txt"
    path_Texts.append(path_Text)
    #print(path_Text)
    
len(path_Texts)
path_Texts[4]

1328

'F:/Environmental Baseline Data/Version 3/Data/HTML Images and Tables/A0U3G1/table-of-contents.txt'

In [8]:
# Here we are not saving or extracting the TOCs yet
# Here we are only checking if the extraction of the TOC is possible or not 
TOC_present = []

txt_path = "F:/Environmental Baseline Data/Version 3/Data/Text/"
TOC_present = []
txt_file_needed = []
no_HTML = 0

for path_Text in path_Texts:
    try:
        file_handle = open(path_Text,"r", encoding="utf8")
        if file_handle.mode == 'r':
            contents = file_handle.read()
            contents = contents.upper()
            if 'TABLE OF CONTENTS' in contents:
                TOC_present.append(1)
            else:
                TOC_present.append(0)
                
    except:
        TOC_present.append('0')
        no_HTML = no_HTML+1
        
len(file_nos)
len(file_names)
len(file_paths)
len(int_postals)
len(TOC_present)


print("HTML File does not exist for {} files".format(no_HTML))

1328

1328

1328

1328

1328

HTML File does not exist for 189 files


In [9]:
df_outlines = pd.DataFrame({'file_nos': file_nos, 
                            'file_names': file_names, 
                            'file_paths' : file_paths, 
                            'int_postals': int_postals, 
                            #'outline_present': outline_present,
                            'TOC_present': TOC_present})
df_outlines.head(5)

,file_nos,file_names,file_paths,int_postals,TOC_present
0,1,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20300,1
1,1,A0H8C1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20301,0
2,1,A0H8C2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20302,0
3,1,A0H8C3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20303,0
4,1,A0U3G1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,52841,1


### 1 d. Make Chunks 

Defination of a Chunk: <br>
    1 Each chunk of the pdf should have consecutive PDF files <br>
    2 Each chunk of the files can have only one TOC present  <br>
    3 The first file of a chunk should be the file to have the outline/TOC present <br>
    

In [10]:
chunk_nos = [1]
chunk = 1
consecutive_postal_codes = [0]

for i in range(1, len(int_postals)):
    new_chunk_found = 0
    
    # If conditions for new chunk based on consecutine postal codes - Ref 1 
    if int_postals[i] != int_postals[i-1] + 1:
        new_chunk_found = 1
        chunk = chunk + 1
        
        
    ## If conditions for new chunk based on outline present - Ref 2 and 3    
    #if outline_present[i] == 1 and outline_present[i-1] ==1 and new_chunk_found == 0: 
    #    new_chunk_found = 1
    #    chunk = chunk + 1
        
    ## If conditions for new chunk based on TOC present - Ref 2 and 3    
    if TOC_present[i] == 1 and new_chunk_found == 0: 
        new_chunk_found = 1
        chunk = chunk + 1
        
    ### Chunk not found based on any of the above conditions
    chunk_nos.append(chunk)
        
    if int_postals[i] == int_postals[i-1] + 1:
        consecutive_postal_codes.append(1)
    else:
        consecutive_postal_codes.append(0)
        
len(chunk_nos)
len(consecutive_postal_codes)
chunk

1328

1328

286

In [11]:
df_outlines['consecutive_postals'] = consecutive_postal_codes
df_outlines['chunk_nos'] = chunk_nos
df_outlines.head(5)

,file_nos,file_names,file_paths,int_postals,TOC_present,consecutive_postals,chunk_nos
0,1,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20300,1,0,1
1,1,A0H8C1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20301,0,1,1
2,1,A0H8C2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20302,0,1,1
3,1,A0H8C3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20303,0,1,1
4,1,A0U3G1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,52841,1,0,2


### Check for Outlines Present 

In [12]:
# Here we are not saving or extracting the Outlines yet
# Here we are only checking if the extraction of the outline is possible or not 
outline_present = []

for pdf_path in pdf_paths:
    if 'Thum' in pdf_path:
        continue
        
    try:
        pdfread = p2.PdfFileReader(pdf_path)
        s = pdfread.outlines
        len_s = len(s)
    except:
        outline_present.append(0) # Error in reading the outline
        
    if len_s > 0:
        outline_present.append(1)
    elif len_s == 0:
        outline_present.append(0)
    
    # Error Code for s (=pdfraed.outlines) is -999
    # the negative number allows exclusivity from the if and the elif conditions above
    len_s = -999 

In [13]:
len(file_names)
len(file_paths)
len(outline_present)

1328

1328

1328

In [24]:
df_outlines['outline_present'] = outline_present
df_outlines.head(5)
#df_outlines.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Outline_Present_htmlTOC.csv")

,file_nos,file_names,file_paths,int_postals,TOC_present,consecutive_postals,chunk_nos,outline_present
0,1,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20300,1,0,1,1
1,1,A0H8C1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20301,0,1,1,1
2,1,A0H8C2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20302,0,1,1,0
3,1,A0H8C3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20303,0,1,1,0
4,1,A0U3G1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,52841,1,0,2,0


# 2. Extract Chapters from PDFs 

### 2 a. Table Extraction by TOC 

In [25]:

def find_page_no(look_file_path, keyword):
    """ Given the Keyword and the PDF file, this function returs an array of 
    the page numbers on which the keyword was found"""
    object = p2.PdfFileReader(look_file_path)
    # get number of pages
    NumPages = object.getNumPages() 
    
    # chapter_key_index is set as 1 because we dont want to search for"0 Before First Chapter"
    # the array chapter_key_pages has first element 0 for the same reason
    chapter_key_index = 1
    chapter_key_page_Nos = [0]
    Flag_TOC_over = 0
    
    page_found = [-999]
    atleast_found_once = 0
    # extract text and do the search
    for i in range(0, NumPages):
        PageObj = object.getPage(i)
        #print("this is page " + str(i)) 
        try:
            Text = PageObj.extractText()
        except:
            Text = "Text could not be extracted"
        Text = Text.lower()
        Text = Text.replace("\n", "")
        # Text = Text
        # print (Text)
            
        #ResSearch = re.search(chapter_keys[chapter_key_index], Text)
        key = keyword.lower()
        if (key in Text):
            page_found.append(i)
            atleast_found_once = 1
            
    if atleast_found_once == 1:
        page_found = page_found[1:]
    return(page_found)
            

"""
def find_page_no(look_file_path, keyword):
    object = p2.PdfFileReader(look_file_path)
    # get number of pages
    NumPages = object.getNumPages() 
    
    # chapter_key_index is set as 1 because we dont want to search for"0 Before First Chapter"
    # the array chapter_key_pages has first element 0 for the same reason
    chapter_key_index = 1
    chapter_key_page_Nos = [0]
    Flag_TOC_over = 0
    
    # extract text and do the search
    for i in range(0, NumPages):
        PageObj = object.getPage(i)
        #print("this is page " + str(i)) 
        Text = PageObj.extractText()
        Text = Text.lower()
        Text = Text.replace("\n", "")
        # Text = Text
        # print (Text)
            
        #ResSearch = re.search(chapter_keys[chapter_key_index], Text)
        key = keyword.lower()
        if (key in Text):
            return i, str(look_file_path.split('/')[-1])
        
    return(-999, "") """

'\ndef find_page_no(look_file_path, keyword):\n    object = p2.PdfFileReader(look_file_path)\n    # get number of pages\n    NumPages = object.getNumPages() \n    \n    # chapter_key_index is set as 1 because we dont want to search for"0 Before First Chapter"\n    # the array chapter_key_pages has first element 0 for the same reason\n    chapter_key_index = 1\n    chapter_key_page_Nos = [0]\n    Flag_TOC_over = 0\n    \n    # extract text and do the search\n    for i in range(0, NumPages):\n        PageObj = object.getPage(i)\n        #print("this is page " + str(i)) \n        Text = PageObj.extractText()\n        Text = Text.lower()\n        Text = Text.replace("\n", "")\n        # Text = Text\n        # print (Text)\n            \n        #ResSearch = re.search(chapter_keys[chapter_key_index], Text)\n        key = keyword.lower()\n        if (key in Text):\n            return i, str(look_file_path.split(\'/\')[-1])\n        \n    return(-999, "") '

In [26]:
def path_pdf_to_text(path_pdf):
    """ This functions helps in converting the PDF file path to the Text File Path
    Input -> Path of the PDF File
    Output -> Path of the Text File, considering that the text file will be relkatively 
              in the same folder
              """
    path_Text = path_pdf.replace(" .pdf", "")
    if ".pdf" in path_Text:
        path_Text = path_pdf.replace(".pdf", "")
    #print(path_Text)
    path_Text = path_Text.replace("/PDF", "/HTML Images and Tables")
    path_Text = path_Text + "/table-of-contents.txt"
    path_Texts.append(path_Text)
    #print(path_Text)
    return (path_Text)
    

In [27]:
def head_clean(head):
    """ This functions helps in removing the textx with a lot of consecutive dots in the string 
    
    Input -> String of the chapter name 
    Eg. 1.5. Environment-Socio Assessment..................................... 34
    
    Output -> String of the chapter name which would be cleaned and can be used to extract as the 
    keys to search. 
    Eg. 1.5. Environment-Socio Assessment
    """
    
    index = 0
    for i in range(len(head)):
        if (head[i] == '.') and (head[i+1] == '.') and (head[i+2] =='.'):
            index = i
            break
        
    head_clean = head[:i]
    return head_clean

In [28]:
# Functtion that runs recursively to extract the outlines of the PDF files
def extract_chapter_keys(path_pdf):
    """This functions gives all the chapter keys when you give the path of the pdf as an input
    This functions works when
       - PDF is converted to HTML
       - We have extracted the Table of Contents from the HTML files
       
    Input -> Path of the PDF files
    Output -> The list of the chapter keys which woudl be used to search 
    
    This functions uses path_pdf_to_text() and head_clean() as dependent functions"""
    
    path_Text = path_pdf_to_text(path_pdf)
    print(path_Text)
    with open(path_Text) as json_file:
        data = json.load(json_file) #, encoding='utf8', errors='ignore')
        raw_TOC_text = data['raw_text'] #currently the array has a lot of strings which are not main headers 
        if len(raw_TOC_text) < 1:
            print("No TOC Found")
    
    
    chapter_keys = ["0 Before First Chapter"] 
    Flag_Appendix_Found = 0
    for head in raw_TOC_text:
        head = head.strip()
        head = re.sub(' +', ' ',head)
        if head[0].isdigit() and len(head) > 5: # assuming all the headings in the TOC are numbered
            head = head_clean(head) # removing soo many fillers '.' in the string
            
            ##########################################
            # Here, we night need to set a threshold (On the number of dots) value depending on the PDF and its TOC 
            ##########################################
            
            threshold = 2
            if head.count('.') < threshold:
                head = head.strip()
                chapter_keys.append(head)
                
            # Here we check for Appendix 
        if "Appendix" in head or "APPENDIX" in head:
            chapter_keys.append("99 Appendix")
            Flag_Appendix_Found = 1
            break
                
    if Flag_Appendix_Found == 0:
        chapter_keys.append("99 Appendix")
    return chapter_keys

In [29]:
def extract_TOC_chapters(file_pdf_path, current_chunk, PDF_names, 
                         PDF_locations, Chunk_No, Chapter_numbers, 
                         Chapter_texts, Chapter_keys):
    """ This function allows the use of fucntions to extract arrays of the chapter keys 
    which would be useful in extracting arrays of similar lengths to be used in Dataframes
    """
    index = -1
    chapter_keys = extract_chapter_keys(file_pdf_path)
    for chapter_key in chapter_keys:
        PDF_names.append(file_pdf_path.split('/')[-1])
        PDF_locations.append(file_pdf_path)
        Chunk_No.append(current_chunk)
        if ' ' in chapter_key:
            index = chapter_key.index(' ')
        Chapter_numbers.append(chapter_key[:index])
        Chapter_texts.append(chapter_key[index+1:])
    Chapter_keys.extend(chapter_keys)


In [30]:
# function to count the occourance of a keyword  
def count_occorances(lst, x):
    count = 0
    for ele in lst:
        if ele == x:
            count = count + 1
    return count

In [31]:
len(df_outlines)

1328

In [32]:
PDF_names = []
PDF_locations = []
Chunk_No = []
Chapter_numbers = []
Chapter_texts = []
Chapter_keys = []
PDF_Found = []
Page_Found = []

allChaptersDF = pd.DataFrame({'PDF_names' : PDF_names, 
                              'PDF_locations' : PDF_locations,
                              'Chunk_No' : Chunk_No,
                              'Chapter_numbers' : Chapter_numbers, 
                              'Chapter_texts' : Chapter_texts, 
                              'Chapter_keys' : Chapter_keys,
                              'PDF_Found': PDF_Found,
                              'Page_Found' : Page_Found})


In [33]:
TOC_Found_in_Chunks = []
# This array is used to store the information wheather each of the chunk 
# has a TOC or not. 
# The length of this array would be equal to the number of chunks.
# (not equal to the number of chapters)




for current_chunk in range(274,276):
    current_df_outlines = df_outlines[df_outlines.chunk_nos == current_chunk]
    chunk_has_TOC = 0
    chunk_noOutline_hasTOC = 0
    print("Processing chunk  Number {} with {} files".format(current_chunk, len(current_df_outlines)))
    
    TOC_found = 0
    
    PDF_names = []
    PDF_locations = []
    Chunk_No = []
    Chapter_numbers = []
    Chapter_texts = []
    Chapter_keys = []
    PDF_Found = []
    Page_Found = []

    
    for i in range(len(current_df_outlines)):
        # First file in the chunk has Outline 
        if current_df_outlines.iloc[i]['TOC_present'] == 1:
            try:
                extract_TOC_chapters(current_df_outlines.iloc[i]['file_paths'], 
                                     current_chunk, PDF_names, PDF_locations, 
                                     Chunk_No, Chapter_numbers, Chapter_texts,
                                     Chapter_keys)
                #print(len(PDF_locations))
                #print(len(PDF_names))
                #print(len(Chapter_numbers))
                #print(len(Chapter_texts))
                
                if(len(Chapter_numbers) > 1):
                    TOC_found = 1
                    PDF_Found = [[-1]]
                    Page_Found = [[-1]]
                    for ikey in range(1,len(Chapter_keys)):
                        PDF_Found.append([])
                        Page_Found.append([])
                    #print(len(PDF_Found))
                
                chapter_index_search = 1        
                
            except:
                print("Error in finding the TOC")
                continue
                
            
            # Because we want to start searching from the 2nd element 
            # as the first element of the key array is a filler 
            
        if TOC_found == 1:
            
            for chapter_index_search in range(1, len(Chapter_keys)):
                print(i)
                print("search started for file {} for keyword {}".format(current_df_outlines.iloc[i]['file_paths'],
                                                                        Chapter_keys[chapter_index_search]))
                page_found = find_page_no(current_df_outlines.iloc[i]['file_paths'], 
                                          Chapter_keys[chapter_index_search])
                
                if page_found != [-999]:
                    PDF_Found[chapter_index_search].append(current_df_outlines.iloc[i]['file_paths'].split('/')[-1])
                    Page_Found[chapter_index_search].append(page_found)
                    print("Found Chapter name {} at page No. {}".format(Chapter_keys[chapter_index_search], 
                                                                    page_found))
                    chapter_index_search = chapter_index_search +1 
        
    
    while (len(PDF_Found) != len(PDF_names)):
        print("while loop started")
        if len(PDF_Found) < len(PDF_names):
            PDF_Found.append("")
            Page_Found.append("")
        else:
            break
            
    
    TOC_Found_in_Chunks.append(TOC_found) 
           
    df_chapters = pd.DataFrame({'PDF_names' : PDF_names, 
                            'PDF_locations' : PDF_locations,
                            'Chunk_No' : Chunk_No,
                            'Chapter_numbers' : Chapter_numbers, 
                            'Chapter_texts' : Chapter_texts, 
                            'Chapter_keys' : Chapter_keys,
                            'PDF_Found': PDF_Found,
                            'Page_Found' : Page_Found}, dtype= str)
    allChaptersDF = pd.concat([allChaptersDF, df_chapters])
    allChaptersDF.head(5)


Processing chunk  Number 274 with 6 files
F:/Environmental Baseline Data/Version 3/Data/HTML Images and Tables/A6F4Q3/table-of-contents.txt
0
search started for file F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q3.pdf for keyword 2021 NGTL System Expansion Projec
Found Chapter name 2021 NGTL System Expansion Projec at page No. [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 99, 100, 101, 102, 104, 105, 106, 107, 108, 109, 110, 111]
0
search started for file F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q3.pdf for keyword 2021 NGTL System Expansion Projec
Found Chapter name 2021 NGTL System Expansion Projec at page No. [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1

Found Chapter name 2021 NGTL System Expansion Projec at page No. [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260,

,PDF_names,PDF_locations,Chunk_No,Chapter_numbers,Chapter_texts,Chapter_keys,PDF_Found,Page_Found
0,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,0,Before First Chapter,0 Before First Chapter,[-1],[-1]
1,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,2021,NGTL System Expansion Projec,2021 NGTL System Expansion Projec,"[A6F4Q3.pdf, A6F4Q4.pdf, A6F4Q5.pdf, A6F4Q6.pd...","[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1..."
2,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,2021,NGTL System Expansion Projec,2021 NGTL System Expansion Projec,"[A6F4Q3.pdf, A6F4Q4.pdf, A6F4Q5.pdf, A6F4Q6.pd...","[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1..."
3,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,99,Appendix,99 Appendix,[],[]


Processing chunk  Number 275 with 4 files
F:/Environmental Baseline Data/Version 3/Data/HTML Images and Tables/A6F4Q9/table-of-contents.txt
0
search started for file F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q9.pdf for keyword 2021 NGTL System Expansion Project June 201
Found Chapter name 2021 NGTL System Expansion Project June 201 at page No. [182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 266]
0
search started for file F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q9.pdf for keyword 1.0 INTRODUCTION
Found Chapter name 1.0 INTRODUCTION at page No. [3]
0
search started for file F:/Environmental Baseline Data/Version 3/Data/P

2
search started for file F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R1.pdf for keyword 1.0 INTRODUCTION
2
search started for file F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R1.pdf for keyword 2.0 PURPOSE
2
search started for file F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R1.pdf for keyword 3.0 ENVIRONMENTAL PROTECTION PLAN ORGANIZATION
2
search started for file F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R1.pdf for keyword 3.1 Organization
2
search started for file F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R1.pdf for keyword 3.2 Environmental Setting
2
search started for file F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R1.pdf for keyword 3.3 Extent and Limits of the Environmental Protection Plan
2
search started for file F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R1.pdf for keyword 4.0 ENVIRONMENTAL COMPLIANCE
2
search started for file F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R1.pdf for keyword 5

,PDF_names,PDF_locations,Chunk_No,Chapter_numbers,Chapter_texts,Chapter_keys,PDF_Found,Page_Found
0,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,0,Before First Chapter,0 Before First Chapter,[-1],[-1]
1,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,2021,NGTL System Expansion Projec,2021 NGTL System Expansion Projec,"[A6F4Q3.pdf, A6F4Q4.pdf, A6F4Q5.pdf, A6F4Q6.pd...","[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1..."
2,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,2021,NGTL System Expansion Projec,2021 NGTL System Expansion Projec,"[A6F4Q3.pdf, A6F4Q4.pdf, A6F4Q5.pdf, A6F4Q6.pd...","[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1..."
3,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,99,Appendix,99 Appendix,[],[]
0,A6F4Q9.pdf,F:/Environmental Baseline Data/Version 3/Data/...,275,0,Before First Chapter,0 Before First Chapter,[-1],[-1]


In [36]:
len(PDF_Found)
len(PDF_names)
len(Chapter_numbers)

24

24

24

In [37]:
allChaptersDF.head(25)
len(allChaptersDF)
allChaptersDF.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Index chunks 274,275- Chapters by TOC HTML.csv")

,PDF_names,PDF_locations,Chunk_No,Chapter_numbers,Chapter_texts,Chapter_keys,PDF_Found,Page_Found
0,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,0,Before First Chapter,0 Before First Chapter,[-1],[-1]
1,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,2021,NGTL System Expansion Projec,2021 NGTL System Expansion Projec,"[A6F4Q3.pdf, A6F4Q4.pdf, A6F4Q5.pdf, A6F4Q6.pd...","[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1..."
2,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,2021,NGTL System Expansion Projec,2021 NGTL System Expansion Projec,"[A6F4Q3.pdf, A6F4Q4.pdf, A6F4Q5.pdf, A6F4Q6.pd...","[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1..."
3,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,99,Appendix,99 Appendix,[],[]
0,A6F4Q9.pdf,F:/Environmental Baseline Data/Version 3/Data/...,275,0,Before First Chapter,0 Before First Chapter,[-1],[-1]
1,A6F4Q9.pdf,F:/Environmental Baseline Data/Version 3/Data/...,275,2021,NGTL System Expansion Project June 201,2021 NGTL System Expansion Project June 201,[A6F4Q9.pdf],"[[182, 183, 184, 185, 186, 187, 188, 189, 190,..."
2,A6F4Q9.pdf,F:/Environmental Baseline Data/Version 3/Data/...,275,1.0,INTRODUCTION,1.0 INTRODUCTION,"[A6F4Q9.pdf, A6F4R2.pdf]","[[3], [93, 94, 217, 220, 281, 284, 300, 302, 3..."
3,A6F4Q9.pdf,F:/Environmental Baseline Data/Version 3/Data/...,275,2.0,PURPOSE,2.0 PURPOSE,[A6F4Q9.pdf],[[5]]
4,A6F4Q9.pdf,F:/Environmental Baseline Data/Version 3/Data/...,275,3.0,ENVIRONMENTAL PROTECTION PLAN ORGANIZATION,3.0 ENVIRONMENTAL PROTECTION PLAN ORGANIZATION,[A6F4Q9.pdf],[[6]]
5,A6F4Q9.pdf,F:/Environmental Baseline Data/Version 3/Data/...,275,3.1,Organization,3.1 Organization,[A6F4Q9.pdf],[[6]]


28

### Adding Outlines 

For some of the chunks that do not have TOC present in the HTML files might still have outlinmes present. 

In [23]:
for i in range(274,276):
    print(i)

274
275


# Rough Work 

In [64]:
def find_page_no(look_file_path, keyword):
    """ Given the Keyword and the PDF file, this function returs an array of 
    the page numbers on which the keyword was found"""
    object = p2.PdfFileReader(look_file_path)
    # get number of pages
    NumPages = object.getNumPages() 
    
    # chapter_key_index is set as 1 because we dont want to search for"0 Before First Chapter"
    # the array chapter_key_pages has first element 0 for the same reason
    chapter_key_index = 1
    chapter_key_page_Nos = [0]
    Flag_TOC_over = 0
    
    page_found = [-999]
    atleast_found_once = 0
    # extract text and do the search
    for i in range(0, NumPages):
        PageObj = object.getPage(i)
        #print("this is page " + str(i)) 
        try:
            Text = PageObj.extractText()
        except:
            Text = "Text could not be extracted"
        Text = Text.lower()
        Text = Text.replace("\n", "")
        # Text = Text
        # print (Text)
            
        #ResSearch = re.search(chapter_keys[chapter_key_index], Text)
        key = keyword.lower()
        if (key in Text):
            page_found.append(i)
            atleast_found_once = 1
            
    if atleast_found_once == 1:
        page_found = page_found[1:]
    return(page_found)
            

In [136]:
array = [1,2,3]
array = array[1:]
array

[2, 3]

In [151]:
# I dont think I needed this function 
def count_occorances(lst, x):
    count = 0
    for ele in lst:
        if ele == x:
            count = count + 1
    return count

In [141]:
df_outlines[["file_names"]].head(5)

,file_names
0,A0H8C0.pdf
1,A0H8C1.pdf
2,A0H8C2.pdf
3,A0H8C3.pdf
4,A0U3G1.pdf


In [142]:
len(df_outlines)

1328

In [143]:
df_outlines[:1]

,file_nos,file_names,file_paths,int_postals,outline_present,TOC_present,consecutive_postals,chunk_nos
0,1,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20300,1,1,0,1


In [144]:
df_outlines[:1][["file_names"]]

,file_names
0,A0H8C0.pdf


In [145]:
df_outlines[:1][["outline_present"]]
(df_outlines[:1].values([outline_present]))

,outline_present
0,1


TypeError: 'numpy.ndarray' object is not callable

In [146]:
df_outlines.iloc[0]['outline_present']
df_outlines.iloc[0]['outline_present'] == 1

1

True

In [157]:
current_df_outlines = df_outlines[df_outlines.chunk_nos == 3]
type(current_df_outlines.file_paths)
chunk_file_paths = current_df_outlines.file_paths
chunk_size = len(chunk_file_paths)
chunk_size
len(current_df_outlines.file_paths)

pandas.core.series.Series

2

2